In [1]:
import tensorflow as tf
import numpy as np
%matplotlib inline

In [2]:
tf.set_random_seed(777)  # for reproducibility
learning_rate = 0.1

In [3]:
x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]
x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.float32)

In [4]:
X = tf.placeholder(tf.float32, [None, 2])
Y = tf.placeholder(tf.float32, [None, 1])

In [5]:
with tf.name_scope("layer1") as scope:
    W1 = tf.Variable(tf.random_normal([2, 2]), name='weight1')
    b1 = tf.Variable(tf.random_normal([2]), name='bias1')
    layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

    w1_hist = tf.summary.histogram("weights1", W1)
    b1_hist = tf.summary.histogram("biases1", b1)
    layer1_hist = tf.summary.histogram("layer1", layer1)

In [6]:
with tf.name_scope("layer2") as scope:
    W2 = tf.Variable(tf.random_normal([2, 1]), name='weight2')
    b2 = tf.Variable(tf.random_normal([1]), name='bias2')
    hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)

    w2_hist = tf.summary.histogram("weights2", W2)
    b2_hist = tf.summary.histogram("biases2", b2)
    hypothesis_hist = tf.summary.histogram("hypothesis", hypothesis)

In [7]:
# cost/loss function
with tf.name_scope("cost") as scope:
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
                           tf.log(1 - hypothesis))
    cost_summ = tf.summary.scalar("cost", cost)

In [8]:
with tf.name_scope("train") as scope:
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [9]:
# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
accuracy_summ = tf.summary.scalar("accuracy", accuracy)

In [10]:
# Launch graph
with tf.Session() as sess:
    # tensorboard --logdir=./logs/xor_logs
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/xor_logs_r0_01")
    writer.add_graph(sess.graph)  # Show the graph

    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        summary, _ = sess.run([merged_summary, train], feed_dict={X: x_data, Y: y_data})
        writer.add_summary(summary, global_step=step)

        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={
                  X: x_data, Y: y_data}), sess.run([W1, W2]))

    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy],
                       feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)

0 0.710643 [array([[ 0.70269841,  0.77858698],
       [-1.11728501, -0.20518988]], dtype=float32), array([[ 1.80769825],
       [ 0.44572306]], dtype=float32)]
100 0.0253783 [array([[-6.36170626,  7.07269526],
       [ 6.38222504, -7.35125351]], dtype=float32), array([[ 8.07209682],
       [ 8.0960989 ]], dtype=float32)]
200 0.00822237 [array([[-7.17781782,  7.78995323],
       [ 7.20768785, -8.06941032]], dtype=float32), array([[ 10.22089672],
       [ 10.29511356]], dtype=float32)]
300 0.00444398 [array([[-7.56189346,  8.13878059],
       [ 7.59312296, -8.41861057]], dtype=float32), array([[ 11.40473652],
       [ 11.4900198 ]], dtype=float32)]
400 0.00286451 [array([[-7.81769514,  8.37369728],
       [ 7.84908915, -8.65380478]], dtype=float32), array([[ 12.25248814],
       [ 12.3428421 ]], dtype=float32)]
500 0.00202969 [array([[-8.00879765,  8.5504694 ],
       [ 8.04007626, -8.83081436]], dtype=float32), array([[ 12.91898441],
       [ 13.01258183]], dtype=float32)]
600 0.0015263

5100 2.36484e-05 [array([[ -9.9392128 ,  10.38260746],
       [  9.96501827, -10.66657257]], dtype=float32), array([[ 21.62299919],
       [ 21.73971939]], dtype=float32)]
5200 2.23371e-05 [array([[ -9.95891857,  10.40163708],
       [  9.98465443, -10.68564606]], dtype=float32), array([[ 21.73408127],
       [ 21.85097504]], dtype=float32)]
5300 2.11003e-05 [array([[ -9.97841263,  10.42047119],
       [ 10.00408077, -10.70452404]], dtype=float32), array([[ 21.84450912],
       [ 21.96155739]], dtype=float32)]
5400 1.99678e-05 [array([[ -9.99771023,  10.43911457],
       [ 10.02330875, -10.7232151 ]], dtype=float32), array([[ 21.95434189],
       [ 22.07154655]], dtype=float32)]
5500 1.88949e-05 [array([[-10.01681995,  10.45758438],
       [ 10.04234695, -10.74172878]], dtype=float32), array([[ 22.063591  ],
       [ 22.18096924]], dtype=float32)]
5600 1.78965e-05 [array([[-10.03574848,  10.47587872],
       [ 10.06120586, -10.76007462]], dtype=float32), array([[ 22.17229271],
       [

Hypothesis:  [[  1.93623873e-06]
 [  9.99998212e-01]
 [  9.99998450e-01]
 [  1.90694220e-06]] 
Correct:  [[ 0.]
 [ 1.]
 [ 1.]
 [ 0.]] 
Accuracy:  1.0
